In [1]:
import pandas as pd
import numpy as np

In [2]:
games=pd.read_csv(r"video_games.csv")

In [3]:
games

,Title,playerNum,Online,Genre,Publisher,RS,Sales,UP,Console,AgeRating,Year
0,Super Mario 64 DS,1,True,Action,Nintendo,85,4.69,24.95,Nintendo DS,E,2004
1,Lumines: Puzzle Fusion,1,True,Strategy,Ubisoft,89,0.56,14.95,Sony PSP,E,2004
2,WarioWare Touched!,2,True,"Action,Racing / Driving,Sports",Nintendo,81,0.54,22.95,Nintendo DS,E,2004
3,Hot Shots Golf: Open Tee,1,True,Sports,Sony,81,0.49,12.95,Sony PSP,E,2004
4,Spider-Man 2,1,True,Action,Activision,61,0.45,14.95,Nintendo DS,E,2004
...,...,...,...,...,...,...,...,...,...,...,...
942,De Blob,4,True,Action,THQ,82,0.45,16.95,Nintendo Wii,M,2008
943,Sonic Riders: Zero Gravity,4,True,Racing / Driving,Sega,56,0.45,16.95,Nintendo Wii,T,2008
944,Rhythm Heaven,1,True,Action,Nintendo,83,0.45,24.95,Nintendo DS,T,2008
945,Tomb Raider: Underworld,1,True,Action,Eidos,75,0.44,17.95,PlayStation 3,T,2008


In [4]:
game_sugg = games[['Title', 'Genre', 'Console']]
game_sugg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    947 non-null    object
 1   Genre    947 non-null    object
 2   Console  947 non-null    object
dtypes: object(3)
memory usage: 22.3+ KB


In [5]:
# games.dropna(inplace=True)

In [6]:
games["Title"]

0               Super Mario 64 DS
1          Lumines: Puzzle Fusion
2              WarioWare Touched!
3        Hot Shots Golf: Open Tee
4                    Spider-Man 2
                  ...            
942                       De Blob
943    Sonic Riders: Zero Gravity
944                 Rhythm Heaven
945       Tomb Raider: Underworld
946    Advance Wars: Days of Ruin
Name: Title, Length: 947, dtype: object

In [7]:
games.isnull().sum()

Title          0
playerNum      0
Online         0
Genre          0
Publisher    171
RS             0
Sales          0
UP             0
Console        0
AgeRating      0
Year           0
dtype: int64

In [8]:
# games.dropna(inplace=True)

In [9]:
# games.duplicated().value_counts()

In [10]:
# games=games.drop_duplicates(keep="first", inplace=False)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
vector_game = TfidfVectorizer(min_df = 3,
                         max_features = None,
                         strip_accents = 'unicode',
                         analyzer = 'word',
                         token_pattern = '\w{2,}',
                         ngram_range = (1,3),
                         stop_words = 'english')

In [13]:
tfv_matrix=vector_game.fit_transform(games["Title"])
tfv_matrix.shape

(947, 518)

In [14]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [15]:
sig_game=sigmoid_kernel(tfv_matrix, tfv_matrix)

In [16]:
sig_game

array([[0.76240373, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76240373, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       ...,
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76240373,
        0.76159416],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76240373]])

In [17]:
sig_game.shape

(947, 947)

In [18]:
# index = pd.Series(games.index, index=games['Title']).drop_duplicates()
index = pd.Series(games.index, index=games['Title'])

In [19]:
index

Title
Super Mario 64 DS               0
Lumines: Puzzle Fusion          1
WarioWare Touched!              2
Hot Shots Golf: Open Tee        3
Spider-Man 2                    4
                             ... 
De Blob                       942
Sonic Riders: Zero Gravity    943
Rhythm Heaven                 944
Tomb Raider: Underworld       945
Advance Wars: Days of Ruin    946
Length: 947, dtype: int64

In [20]:
def recommend_game(title, sig_kern=sig_game):
    indx = index[title]
#     print(indx)

    if isinstance(indx, list):
        sigmoid_score = list(enumerate(sig_kern[indx[0]]))
    else:
        sigmoid_score = list(enumerate(sig_kern[indx]))
    
#     print(sigmoid_score)
    sigmoid_score = sorted(sigmoid_score, key = lambda x:x[1], reverse = True)
    sigmoid_score = sigmoid_score[1:4]
    game_found = [i[0] for i in sigmoid_score]
    return games.iloc[game_found]

In [21]:
recommend_game('De Blob')

,Title,playerNum,Online,Genre,Publisher,RS,Sales,UP,Console,AgeRating,Year
1,Lumines: Puzzle Fusion,1,True,Strategy,Ubisoft,89,0.56,14.95,Sony PSP,E,2004
2,WarioWare Touched!,2,True,"Action,Racing / Driving,Sports",Nintendo,81,0.54,22.95,Nintendo DS,E,2004
3,Hot Shots Golf: Open Tee,1,True,Sports,Sony,81,0.49,12.95,Sony PSP,E,2004


In [22]:
recommend_game('Spider-Man 2')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [23]:
recommend_game('Madden NFL 2005')

,Title,playerNum,Online,Genre,Publisher,RS,Sales,UP,Console,AgeRating,Year
434,Madden NFL 08,4,True,Sports,EA,85,2.32,9.95,X360,E,2007
467,Madden NFL 08,4,True,Sports,EA,81,0.94,9.95,PlayStation 3,T,2007
475,Madden NFL 08,4,True,Sports,EA,76,0.86,12.95,Nintendo Wii,M,2007
